In [1]:
import csv
import pandas as pd
import numpy as np
import random
import re
import functools

In [2]:
file = '~/Downloads/report1596060698882.csv'

In [3]:
df = pd.read_csv(file)

In [4]:
df['Close Date'] = pd.to_datetime(df['Close Date'])

In [5]:
df['Close Date Year'] = pd.DatetimeIndex(df['Close Date']).year

In [6]:
df.head()

,Opportunity Name,Won/Lost Reason Add. Info,"Lost Reason ""Other"" Detail",Close Date,Sales Amount,Current Quota Credit Units,# of Units for Migration,# of Association Units for Migration,Need,Maintenance / Mobile inspections,Current Situation,Total Units Managed,Won/Loss Reason,Close Date Year
0,Henderson Management & Real Estate,NaN,NaN,2009-07-31,8680.0,0.0,NaN,NaN,NaN,NaN,NaN,0,Feature,2009
1,Marceau Real Estate- 200,NaN,NaN,2009-09-30,2800.0,NaN,NaN,NaN,NaN,NaN,NaN,0,Feature,2009
2,Select Portfolio Servicing,NaN,NaN,2009-09-30,4000.0,NaN,NaN,NaN,NaN,NaN,NaN,0,Feature,2009
3,J and K Realty Services-200 units,NaN,NaN,2009-09-30,2800.0,NaN,NaN,NaN,NaN,NaN,NaN,0,Feature,2009
4,California Development Authority-1327 units,NaN,NaN,2009-09-30,13933.5,NaN,NaN,NaN,NaN,NaN,NaN,0,Feature,2009


In [7]:
df.loc[df[['Current Quota Credit Units' ,'# of Units for Migration', 'Total Units Managed']].max(axis=1) == 0, 'EST_UNITS'] = round(df['Sales Amount']/17,0)
df.loc[df[['Current Quota Credit Units' ,'# of Units for Migration', 'Total Units Managed']].max(axis=1) > 0, 'EST_UNITS'] = df[['Current Quota Credit Units' ,'# of Units for Migration', 'Total Units Managed']].max(axis=1)

In [8]:
target_cols = ['Lost Reason "Other" Detail' ,'Need', 'Maintenance / Mobile inspections', 'Current Situation']
projects = ['inspections', 'turn_board', 'maintenance_tech', 'work_orders', 'gen_maintenance']
project_words = {
    'inspections': ['inspection'],
    'turn_board': ['make read', 'unit turn', 'turn board', 'turnboard'],
    'maintenance_tech' : ['maintenance tech', 'inhouse maintenance', 'maintenance staff'],
    'work_orders': ['work order'],
    'gen_maintenance' : ['maintenance']
}
masks = {}

In [9]:
for project in projects:
    mask_name = 'mask_{}'.format(project)
    mask = functools.reduce(np.logical_or,[df[i].str.contains(r'{}'.format('|'.join(project_words[project])), flags=re.IGNORECASE, regex=True, na=False) for i in target_cols])
    masks[mask_name] = mask
masks['all'] = functools.reduce(np.logical_or,[df[i].str.contains(r'{}'.format('|'.join(map(str, sum(project_words.values(), [])))), flags=re.IGNORECASE, regex=True, na=False) for i in target_cols])

In [10]:
for mask in masks.keys():
    df[['Close Date Year', 'EST_UNITS']].loc[masks[mask]].groupby(['Close Date Year']).sum().to_csv('~/Downloads/csv_{}.csv'.format(mask))
    print(mask+'\n', df[['Close Date Year', 'EST_UNITS']].loc[masks[mask]].groupby(['Close Date Year']).sum())

mask_inspections
                  EST_UNITS
Close Date Year           
2013                 794.0
2014                5350.0
2015               49079.0
2016               24100.0
2017               41579.0
2018               27516.0
2019               49861.0
2020               28433.0
mask_turn_board
                  EST_UNITS
Close Date Year           
2015                3265.0
2016                1025.0
2020               21500.0
mask_maintenance_tech
                  EST_UNITS
Close Date Year           
2013                3500.0
2014                1835.0
2015                9557.0
2016                5044.0
2017               16752.0
2018                6056.0
2019               20385.0
2020                5911.0
mask_work_orders
                  EST_UNITS
Close Date Year           
2013                4877.0
2014                8649.0
2015               41078.0
2016               20049.0
2017               33276.0
2018               52387.0
2019               37912.0
2020  

In [11]:
df[['Close Date Year', 'EST_UNITS']].groupby(['Close Date Year']).sum()

,EST_UNITS
Close Date Year,
2009,3474.0
2010,29280.0
2011,19649.0
2012,94284.0
2013,137635.0
2014,95569.0
2015,115617.0
2016,106765.0
2017,1091854.0
